In [1]:
import time
start = time.time()

import random
import math as m
import numpy as np
import tensorflow as tf

tf.config.run_functions_eagerly(True)

physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
import tensorflow_probability as tfp
import os
import sys

parent_directory = '/project/ag-seitz/users/sijin/master_thesis/MCMC/affine'
sys.path.append(parent_directory)

from affine.affine.affine import *
# from scipy import interpolate
# from cosmopower import cosmopower_NN


# checking that we are using a GPU
device = 'gpu:0' if tf.test.is_gpu_available() else 'cpu'
print('using', device, 'device \n')

import matplotlib.pyplot as plt


2024-06-15 21:51:29.782910: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
using gpu:0 device 



2024-06-15 21:51:42.083044: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /device:GPU:0 with 4486 MB memory:  -> device: 0, name: Quadro P2000, pci bus id: 0000:61:00.0, compute capability: 6.1


In [2]:
data_np = np.loadtxt("/home/s/Sijin.Chen/course/from_data_to_insights/tutorial08/xi_measurement_tutorial8.dat")
print(data_np.shape)
theta_np = data_np[:, 0]
xi_np = data_np[:, 1]
err_np = data_np[:, 2]
err_cov_np = np.diag(err_np)

(20, 3)


In [3]:
theta_tf = tf.convert_to_tensor(theta_np)
xi_tf = tf.convert_to_tensor(xi_np)
err_tf = tf.convert_to_tensor(err_np)



2024-06-15 21:51:42.715201: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4486 MB memory:  -> device: 0, name: Quadro P2000, pci bus id: 0000:61:00.0, compute capability: 6.1


In [4]:
# target log prior
@tf.function
def log_prior(params):
    A, B, alpha_B = params
    if -10.0 <= A <= 10.0 and -10.0 <= B <= 10.0 and 0.0 <= alpha_B <=0.4 :
        return 0.0
    return -np.inf

In [5]:
# target log likelihood
@tf.function
def log_likelihood(params, theta, xi, err_cov):
    A, B, alpha_B = params
    xi_model = A*theta **0.5 + B*theta **alpha_B
    diff = xi_model - xi      
    result = -0.5 * diff.T @ np.linalg.inv(err_cov) @ diff 
    
    return result

In [6]:
def log_posterior(params, theta, xi, err_cov):
    
    A, B, alpha_B = params
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, theta, xi, err_cov)


In [7]:
# run the sampler
total_steps = 5000
nwalkers = 500
ndim = 3
walkers1 = tf.random.normal([nwalkers, 3], 0., 1.)
walkers2 = tf.random.normal([nwalkers, 3], 0., 1.)
current_state = [walkers1, walkers2]



with tf.device(device):
    #chain, logp_chain = affine.affine_sample(log_posterior, total_steps, current_state, args=[])
    chain = affine_sample(log_posterior, total_steps, current_state, args=[theta_np, xi_np, err_cov_np])


ValueError: too many values to unpack (expected 3)

In [ ]:
# def log_posterior(params, data, inv_cov):
    
#     lp = log_prior(params, params.shape[0])
#     posterior = -np.inf * tf.ones(params.shape[0])
    
#     mask = tf.math.is_finite(lp)
#     indices = tf.where(mask)
#     likelihood = log_likelihood(params[mask], data, inv_cov)
    
#     posterior = tf.tensor_scatter_nd_update(posterior, indices, tf.add(likelihood, lp[mask]))
#     return posterior